## Eukaryotic Cellulases

* To see the plot, **run this notebook** by clicking on the arrow symbol above (it will take a couple of seconds to run)

In [1]:
import panel as pn
import plotly.express as px
import pandas as pd

pn.extension('plotly')

# load data
red_df = pd.read_csv('tSNE_cellulase.csv')

# Define the function to update the plot
def update_plot(search_value_cellulases, search_value_species):
    filtered_df = red_df

    if search_value_cellulases:
        search_terms = [term.strip() for term in search_value_cellulases.split(',')]
        for term in search_terms:
            filtered_df = filtered_df[filtered_df['cellulases'].apply(lambda x: term in x)]

    if search_value_species:
        filtered_df = filtered_df[filtered_df['label'].str.contains(search_value_species, case=False, na=False)]

    fig = px.scatter(
        filtered_df, x='PC1', y='PC2', #title=f'{method} plot',
        labels={'PC1': f'tSNE 1', 'PC2': f'tSNE 2'}, 
        hover_data={'clade': True, 'num_cellulases': True, 'cellulases': True, 'label': True, 'PC1': False, 'PC2': False},
        width=1000, height=1000,
        size='num_cellulases',
        color='clade',
        color_discrete_sequence=px.colors.qualitative.D3
    )
    return fig

# Create interactive widgets
search_bar_cellulases = pn.widgets.TextInput(name='Search Cellulases', placeholder='Enter cellulases...')
search_bar_species = pn.widgets.TextInput(name='Search Species', placeholder='Enter species...')

# Bind the function and widgets
@pn.depends(search_bar_cellulases.param.value, search_bar_species.param.value)
def get_plot(search_value_cellulases, search_value_species):
    return update_plot(search_value_cellulases, search_value_species)

# Layout
layout = pn.Column(pn.Row(search_bar_cellulases, search_bar_species), get_plot)


## tSNE

In [2]:
# Serve the Panel app
layout.servable()

Column
    [0] Row
        [0] TextInput(name='Search Cellulases', placeholder='Enter cellulases...')
        [1] TextInput(name='Search Species', placeholder='Enter species...')
    [1] ParamFunction(function, _pane=Plotly, defer_load=False)